# Model Live Testing


This notebook allows you to **test the flood-detection models on real Sentinel-1 SAR data**.

To use it:

1. Prepare two GeoTIFF files for the **same area and same date**:  
   - one **VV** band file (e.g., `*_VV.tif`)  
   - one **VH** band file (e.g., `*_VH.tif`)
2. Make sure you **update the model paths** in the code if your `.keras`, `.pkl`, or `.joblib` files are stored in a different Drive folder.
3. Choose the model you want to test from the **Model Selection** section (CNN, Random Forest, or SVM).
4. Upload both VV and VH files in the **Upload Sentinel-1 GeoTIFFs** area.
5. The app will automatically:
   - convert VV/VH into 6 SAR-based features,  
   - split the scene into 256×256 tiles,  
   - apply the selected model to each tile,  
   - compute the flood probability map,  
   - and provide a final **scene-level flood decision**.
6. Review the outputs:
   - model prediction (flooded / non-flooded)  
   - tile statistics  
   - **heatmap** of probabilities (blue = low, red = high)



In [8]:
# Model Live Testing - installs

!pip install streamlit -q
!pip -q install "rasterio>=1.4.2" "scipy" "pillow"
!pip -q install -U "numpy>=2.0" "ml-dtypes>=0.5.0"
!pip -q install -U rasterio tifffile pillow scipy matplotlib
!pip -q install -U "tensorflow==2.19.0" "tf-keras==2.19.0" "tensorflow-text==2.19.0"


import base64, wave
import streamlit.components.v1 as components
!pkill -f streamlit
import time
time.sleep(3)
print("✅ تم إيقاف تطبيقات Streamlit")

!ps aux | grep streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 43.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.5 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.5 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.5 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.5 which is incompatible.
numba 0.60.0 requires numpy

In [9]:
from google.colab import drive
drive.mount('/content/drive')

import os

paths = {
    "CNN (Deep Learning)": "/content/drive/MyDrive/sen12flood_npz_full_6feats/best_model.keras",
    "Random Forest": "/content/drive/MyDrive/best_rf_model.pkl",
    "SVM": "/content/drive/MyDrive/sen12flood_npz_full_6feats/svm_results/svm_rbf_approx.joblib",
}

for name, p in paths.items():
    print(name, "→", "OK ✅" if os.path.exists(p) else "NOT FOUND ❌")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CNN (Deep Learning) → OK ✅
Random Forest → OK ✅
SVM → OK ✅


In [15]:
%%writefile simple_flood_app.py
import streamlit as st
import numpy as np
import io, base64, wave
from typing import Tuple, List
from scipy.ndimage import uniform_filter
from rasterio.io import MemoryFile
import matplotlib.pyplot as plt
from tensorflow import keras
import joblib
import streamlit.components.v1 as components

# =========================
# Page configuration
# =========================
st.set_page_config(page_title="Flood Detection (S1)", page_icon="🌊", layout="centered")
st.title("🌊 Flood Detection System")
st.caption(
    "Upload two GeoTIFF files (VV and VH) captured after the event. "
    "The app converts them into 6 channels and uses the selected model (256×256×6 → 1) "
    "to decide whether the scene is flooded or not."
)
st.markdown("---")

# =========================
# Model file paths (CHANGE IF NEEDED)
# =========================
MODEL_PATHS = {
    "CNN (Deep Learning)": "/content/drive/MyDrive/sen12flood_npz_full_6feats/best_model.keras",
    "Random Forest": "/content/drive/MyDrive/best_rf_model.pkl",
    "SVM": "/content/drive/MyDrive/sen12flood_npz_full_6feats/svm_results/svm_rbf_approx.joblib",
}

# =========================
# Model selection
# =========================
st.subheader("⚙️ Model Selection")
model_choice = st.radio(
    "Choose which model you want to use for live testing:",
    list(MODEL_PATHS.keys()),
    index=0,
)

st.info(f"You selected: **{model_choice}**")

# =========================
# Load models (cached)
# =========================
@st.cache_resource(show_spinner=True)
def load_cnn_model(path: str):
    return keras.models.load_model(path)

@st.cache_resource(show_spinner=True)
def load_sklearn_model(path: str):
    """
    Loads classical ML models.

    - For SVM (svm_rbf_approx.joblib): returns the full tuple (scaler, rbf, clf)
    - For other models (e.g. Random Forest): returns the estimator itself
    """
    import pickle, os

    basename = os.path.basename(path).lower()

    try:
        obj = joblib.load(path)
    except Exception:
        with open(path, "rb") as f:
            obj = pickle.load(f)

    #  SVM case: keep the full tuple
    if "svm" in basename:
        return obj   # (scaler, rbf, calibrated_svm)

    #  Other models (e.g. Random Forest)
    if isinstance(obj, (list, tuple)) and len(obj) > 0:
        return obj[0]

    if isinstance(obj, dict):
        for key in ["model", "clf", "estimator", "rf", "classifier"]:
            if key in obj:
                return obj[key]

    return obj

def get_selected_model(choice: str):
    """Return the loaded model object based on the user choice."""
    path = MODEL_PATHS[choice]
    if "CNN" in choice:
        return load_cnn_model(path)
    else:
        return load_sklearn_model(path)

try:
    model = get_selected_model(model_choice)
    model_loaded_ok = True
except Exception as e:
    st.error(f"Error loading **{model_choice}** model from `{MODEL_PATHS[model_choice]}`: {e}")
    model = None
    model_loaded_ok = False

# =========================
# Hidden audio helpers
# =========================
def _html_audio_hidden_from_bytes(b: bytes, loop: bool = False, autoplay: bool = True, height: int = 0):
    b64 = base64.b64encode(b).decode("ascii")
    loop_attr = " loop" if loop else ""
    autoplay_attr = " autoplay" if autoplay else ""
    components.html(
        f"""
        <audio{autoplay_attr}{loop_attr} style="display:none">
          <source src="data:audio/wav;base64,{b64}" type="audio/wav">
        </audio>
        """,
        height=height,
    )

def generate_alarm_wav(seconds: float = 2.5, rate: int = 22050) -> io.BytesIO:
    buf = io.BytesIO()
    with wave.open(buf, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(rate)

        beep_freq = 900.0
        beep_dur = 0.15
        gap_dur  = 0.25
        total_dur = seconds

        pattern = []
        elapsed = 0.0
        while elapsed < total_dur:
            t_beep = np.linspace(0, beep_dur, int(rate * beep_dur), False)
            tone = 0.6 * np.sin(2 * np.pi * beep_freq * t_beep)
            fade_len = max(1, int(0.02 * rate))
            if tone.size >= 2 * fade_len:
                tone[:fade_len] *= np.linspace(0.0, 1.0, fade_len)
                tone[-fade_len:] *= np.linspace(1.0, 0.0, fade_len)
            pattern.append(tone); elapsed += beep_dur

            if elapsed < total_dur:
                gap = np.zeros(int(rate * gap_dur))
                pattern.append(gap); elapsed += gap_dur

        snd = np.concatenate(pattern, axis=0)
        snd = np.clip(snd, -0.99, 0.99)
        wf.writeframes((snd * 32767).astype(np.int16).tobytes())

    buf.seek(0)
    return buf

def generate_all_clear_wav(rate: int = 22050) -> io.BytesIO:
    def tone(f_hz, sec, amp=0.5):
        t = np.linspace(0, sec, int(rate * sec), False)
        snd = amp * np.sin(2 * np.pi * f_hz * t)
        fade_len = max(1, int(0.03 * rate))
        env = np.ones_like(snd)
        env[:fade_len] *= np.linspace(0.0, 1.0, fade_len)
        env[-fade_len:] *= np.linspace(1.0, 0.0, fade_len)
        return snd * env

    seg1 = tone(880.0, 0.55, 0.5)
    pause = np.zeros(int(rate * 0.08))
    seg2 = tone(660.0, 0.55, 0.5)
    snd = np.concatenate([seg1, pause, seg2], axis=0)
    snd = np.clip(snd, -0.99, 0.99)

    buf = io.BytesIO()
    with wave.open(buf, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(rate)
        wf.writeframes((snd * 32767).astype(np.int16).tobytes())
    buf.seek(0)
    return buf

def play_alarm_loop_hidden():
    _html_audio_hidden_from_bytes(generate_alarm_wav().getvalue(), loop=True, autoplay=True)

def play_all_clear_once_hidden():
    _html_audio_hidden_from_bytes(generate_all_clear_wav().getvalue(), loop=False, autoplay=True)

# =========================
# SAR → dB → 6 features
# =========================
def to_db(x: np.ndarray, eps: float = 1e-6) -> np.ndarray:
    x = np.clip(x, eps, None)
    out = 10.0 * np.log10(x)
    out[~np.isfinite(out)] = 0.0
    return out.astype(np.float32)

def lee_filter(img: np.ndarray, size: int = 5) -> np.ndarray:
    img = img.astype(np.float32)
    mean = uniform_filter(img, size=size)
    mean_sq = uniform_filter(img**2, size=size)
    var = mean_sq - mean**2
    overall_var = np.nanmean(var)
    w = var / (var + overall_var + 1e-6)
    return (mean + w * (img - mean)).astype(np.float32)

def mk_6_features(vv_db: np.ndarray, vh_db: np.ndarray) -> np.ndarray:
    vv_lin = 10.0 ** (vv_db / 10.0)
    vh_lin = 10.0 ** (vh_db / 10.0)
    feats = np.stack([
        vv_db,                    # 1) VV dB
        vh_db,                    # 2) VH dB
        vv_db + vh_db,            # 3) VV + VH (dB)
        vv_db - vh_db,            # 4) VV - VH (dB)
        vv_lin / (vh_lin + 1e-6), # 5) VV/VH (linear)
        vv_lin * vh_lin           # 6) VV*VH (linear)
    ], axis=0).astype(np.float32)
    feats = np.nan_to_num(feats, nan=0.0, posinf=0.0, neginf=0.0)
    return feats

def normalize_6ch(x6: np.ndarray, mean=None, std=None) -> np.ndarray:
    x = x6.copy().astype(np.float32)
    if mean is not None and std is not None:
        mean = np.array(mean, dtype=np.float32).reshape(6, 1, 1)
        std  = np.array(std, dtype=np.float32).reshape(6, 1, 1)
        std  = np.where(std == 0, 1.0, std)
        return (x - mean) / std
    ch_mean = x.reshape(6, -1).mean(axis=1).reshape(6,1,1)
    ch_std  = x.reshape(6, -1).std(axis=1).reshape(6,1,1)
    ch_std  = np.where(ch_std == 0, 1.0, ch_std)
    return (x - ch_mean) / ch_std

def read_band_from_tif(uploaded) -> Tuple[np.ndarray, dict]:
    raw = uploaded.read()
    with MemoryFile(raw) as memfile:
        with memfile.open() as src:
            arr = src.read(1).astype(np.float32)
            meta = {
                "crs": src.crs,
                "transform": src.transform,
                "height": src.height,
                "width": src.width,
            }
    return arr, meta

def detect_polarization_from_name(name: str):
    n = name.upper()
    if "VV" in n and "VH" in n:
        return "BOTH"
    if "VV" in n:
        return "VV"
    if "VH" in n:
        return "VH"
    return None

def tile_coords(H:int, W:int, tile_size:int=256, stride:int=256) -> List[Tuple[int,int]]:
    coords = []
    for y in range(0, H - tile_size + 1, stride):
        for x in range(0, W - tile_size + 1, stride):
            coords.append((y, x))
    return coords

# =========================
# Unified prediction helper
# =========================
def predict_tile_probs(model_obj, model_type: str, X_tiles: np.ndarray) -> np.ndarray:
    """
    Returns probabilities for each tile for the selected model type.
    - CNN: per-tile standardization + model.predict (sigmoid output)
    - Random Forest: flatten → predict_proba
    - SVM (approx RBF): scaler → rbf → calibrated_svm.predict_proba
    """
    X_tiles = X_tiles.astype(np.float32)

    # ========= CNN =========
    if "CNN" in model_type:
        mu = X_tiles.mean(axis=(1, 2), keepdims=True)
        sd = X_tiles.std(axis=(1, 2), keepdims=True)
        X_std = (X_tiles - mu) / (sd + 1e-8)

        probs = model_obj.predict(X_std, verbose=0).reshape(-1)
        return probs.astype(np.float32)

    N = X_tiles.shape[0]
    X_flat = X_tiles.reshape(N, -1)

    # ========= SVM (approx RBF) =========
    if "SVM" in model_type:
        # توقعي: model_obj = (scaler, rbf, clf)
        if isinstance(model_obj, (list, tuple)) and len(model_obj) == 3:
            scaler, rbf, clf = model_obj
        elif isinstance(model_obj, dict):
            scaler = model_obj.get("scaler")
            rbf    = model_obj.get("rbf")
            clf    = model_obj.get("model")
        else:
            raise ValueError("Unexpected SVM model format. Expected (scaler, rbf, clf).")

        X_scaled = scaler.transform(X_flat)
        X_rbf = rbf.transform(X_scaled)
        probs = clf.predict_proba(X_rbf)[:, 1]
        return probs.astype(np.float32)

    # ========= Random Forest =========
    if hasattr(model_obj, "predict_proba"):
        probs = model_obj.predict_proba(X_flat)[:, 1]
    else:
        if hasattr(model_obj, "decision_function"):
            scores = model_obj.decision_function(X_flat)
            probs = 1.0 / (1.0 + np.exp(-scores))
        else:
            preds = model_obj.predict(X_flat)
            probs = np.asarray(preds, dtype=np.float32)

    return probs.astype(np.float32)

# =========================
# Upload & processing interface
# =========================
st.subheader("📤 Upload Sentinel-1 GeoTIFFs")
files = st.file_uploader(
    "Upload VV and VH (GeoTIFF) files — post-event",
    type=["tif", "tiff"],
    accept_multiple_files=True
)

if files and model_loaded_ok and model is not None:
    vv_file = None
    vh_file = None
    for f in files:
        pol = detect_polarization_from_name(f.name)
        if pol == "VV" and vv_file is None:
            vv_file = f
        elif pol == "VH" and vh_file is None:
            vh_file = f

    if vv_file is None or vh_file is None:
        st.error("Please ensure one file name contains 'VV' and the other 'VH' (e.g., *_VV_.tif and *_VH_.tif).")
    else:
        with st.spinner("Reading VV/VH and generating features..."):
            try:
                vv_lin, meta_vv = read_band_from_tif(vv_file)
                vh_lin, meta_vh = read_band_from_tif(vh_file)

                if vv_lin.shape != vh_lin.shape:
                    st.error(f"VV dimensions ({vv_lin.shape}) ≠ VH dimensions ({vh_lin.shape}). Both must be aligned.")
                    st.stop()

                # Display uploaded bands (linear)
                st.markdown("### 📸 Uploaded Bands")
                col1, col2 = st.columns(2)
                with col1:
                    st.write("**VV Band (linear)**")
                    st.image(vv_lin, caption="VV", use_column_width=True, clamp=True)
                with col2:
                    st.write("**VH Band (linear)**")
                    st.image(vh_lin, caption="VH", use_column_width=True, clamp=True)

                H, W = vv_lin.shape
                vv_db = to_db(vv_lin)
                vh_db = to_db(vh_lin)
                vv_db = lee_filter(vv_db, size=5)
                vh_db = lee_filter(vh_db, size=5)

                feats6 = mk_6_features(vv_db, vh_db)
                feats6 = normalize_6ch(feats6)

                coords = tile_coords(H, W, tile_size=256, stride=256)
                X_tiles = []
                for (y, x0) in coords:
                    patch = feats6[:, y:y+256, x0:x0+256]
                    if patch.shape[1] != 256 or patch.shape[2] != 256:
                        continue
                    patch = np.transpose(patch, (1, 2, 0)).astype(np.float32)
                    X_tiles.append(patch)

                if not X_tiles:
                    st.error("No 256×256 tiles found for the current dimensions.")
                    st.stop()

                X = np.stack(X_tiles, axis=0)

                # === Model prediction (depends on selected model) ===
                probs = predict_tile_probs(model, model_choice, X)
                probs = probs.reshape(-1)

                mean_p = float(np.mean(probs))
                max_p  = float(np.max(probs))
                pct_above = float((probs >= 0.5).mean() * 100.0)

                is_flood = (pct_above >= 20.0) or (mean_p >= 0.5) or (max_p >= 0.8)
                predicted_label = "Flooded" if is_flood else "Non-flooded"

                st.subheader("📊 Scene-level Result")
                if is_flood:
                    st.error(f"## 🚨 FLOOD DETECTED ({model_choice})")
                    play_alarm_loop_hidden()
                else:
                    st.success(f"## ✅ NON-FLOOD AREA ({model_choice})")
                    play_all_clear_once_hidden()

                c1, c2, c3, c4 = st.columns(4)
                c1.metric("Tiles", f"{len(probs)}")
                c2.metric("Mean prob", f"{mean_p:.3f}")
                c3.metric("Max prob", f"{max_p:.3f}")
                c4.metric("% tiles ≥ 0.50", f"{pct_above:.1f}%")

                # =====================================================================
                # Tile probability map
                # =====================================================================
                st.markdown("---")
                st.subheader("🧩 Tile scores (coarse grid)")

                cols = (W - 256) // 256 + 1
                rows = (H - 256) // 256 + 1
                grid = np.full((rows, cols), np.nan, dtype=np.float32)

                idx = 0
                for r in range(rows):
                    for c in range(cols):
                        if idx < len(probs):
                            grid[r, c] = probs[idx]
                            idx += 1

                fig = plt.figure(figsize=(6, 4))
                plt.imshow(
                    grid,
                    vmin=0.0,
                    vmax=1.0,
                    cmap="coolwarm"
                )
                cbar = plt.colorbar(label="Flood probability")
                cbar.ax.set_ylabel("Flood probability", rotation=270, labelpad=15)
                plt.title("Tile probability map (coarse)")
                plt.xlabel("tile col")
                plt.ylabel("tile row")
                st.pyplot(fig)

            except Exception as e:
                st.error(f"An error occurred during reading/processing: {e}")

else:
    if not model_loaded_ok or model is None:
        st.warning(f"Model **{model_choice}** could not be loaded. Check the path or Google Drive mounting.")
    st.info("Choose a model, then upload both GeoTIFF files (VV and VH) to run the analysis.")

Overwriting simple_flood_app.py


In [16]:
!pkill -f streamlit
import time
time.sleep(3)


APP = "/content/simple_flood_app.py"

import os, re, time, subprocess
from google.colab import output

subprocess.run(["pkill","-f","streamlit run"], check=False)
subprocess.run(["pip","-q","install","-U",
                "streamlit==1.39.0","pillow","opencv-python","protobuf==3.20.*"], check=True)

log = open("/content/streamlit.log","w")
p = subprocess.Popen([
    "streamlit","run",APP,
    "--server.port","8501",
    "--server.address","0.0.0.0",
    "--server.headless","true",
    "--server.enableCORS","false",
    "--server.enableXsrfProtection","false",
    "--browser.gatherUsageStats","false",
    "--server.fileWatcherType","none",
], stdout=log, stderr=log, text=True)

print(" Waiting 10s, then opening URL...")
time.sleep(10)
url = output.eval_js("google.colab.kernel.proxyPort(8501)")
print(" OPEN THIS URL:\n", url)
print("\n Logs: !tail -n 200 /content/streamlit.log")


 Waiting 10s, then opening URL...
 OPEN THIS URL:
 https://8501-m-s-2a1ffaeft70dh-d.us-east1-0.prod.colab.dev

 Logs: !tail -n 200 /content/streamlit.log



#  Test Samples Evaluation

This section evaluates the three trained models — **CNN**, **Random Forest**, and **SVM (RBF-Approx.)** — using a small set of **5 curated test samples** taken from the SEN12FLOOD dataset.

Each sample is a **256×256 tile** containing:

* `x_after`: 6 engineered SAR features
* `y`: pixel-level flood mask

For each file, the script:

1. Loads the processed tile.
2. Computes the **truth label** using the flood-pixel ratio (`pos_ratio ≥ 0.01 → flooded`).
3. Applies consistent preprocessing:

   * reshaping (6,256,256 → 256,256,6)
   * per-tile standardization for CNN
4. Runs predictions using:

   * the trained **CNN model**
   * the optimized **Random Forest model**
   * the **RBF-SVM approximate model** (scaler → RBF → calibrated SVM)
5. Prints a full comparison for each sample:

   * **Ground truth**
   * **Predicted label**
   * **Prediction probability** for each model

This helps verify:

* whether the models are loaded correctly
* that preprocessing is consistent
* and how each model behaves on real SAR tiles

> **Reminder:**
> Do not forget to update your model paths (`cnn_path`, `rf_path`, `svm_path`) if your files are stored in a different Drive location.



In [7]:
import numpy as np
from tensorflow import keras
import joblib, pickle
import os

mini_test_paths = [
    "/content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_0020_2019_01_16_0128_0256_A.npz",
    "/content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_46_2019_02_23_0256_0128_A.npz",
    "/content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_54_2019_03_14_0128_0000_A.npz",
    "/content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_58_2019_04_14_0000_0256_A.npz",
    "/content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_0140_2019_03_13_0000_0000_A.npz",
]

print("Found files:")
for p in mini_test_paths:
    print(" -", p, "OK" if os.path.exists(p) else "NOT FOUND")


cnn_path = "/content/drive/MyDrive/sen12flood_npz_full_6feats/best_model.keras"
rf_path  = "/content/drive/MyDrive/best_rf_model.pkl"
svm_path = "/content/drive/MyDrive/sen12flood_npz_full_6feats/svm_results/svm_rbf_approx.joblib"

# CNN
cnn_model = keras.models.load_model(cnn_path)

# Random Forest
try:
    rf_model = joblib.load(rf_path)
except:
    with open(rf_path, "rb") as f:
        rf_model = pickle.load(f)

# SVM
try:
    svm_obj = joblib.load(svm_path)
except:
    with open(svm_path, "rb") as f:
        svm_obj = pickle.load(f)

if isinstance(svm_obj, (list, tuple)) and len(svm_obj) == 3:
    svm_scaler, svm_rbf, svm_clf = svm_obj
else:
    svm_scaler = svm_obj["scaler"]
    svm_rbf    = svm_obj["rbf"]
    svm_clf    = svm_obj["model"]

def predict_cnn(model, X):
    prob = model.predict(X, verbose=0)[0][0]
    pred = int(prob >= 0.5)
    return pred, float(prob)

def predict_rf(model, X, threshold=0.48):
    N = X.shape[0] if X.ndim == 4 else 1
    X_flat = X.reshape(1, -1).astype(np.float32)
    prob = model.predict_proba(X_flat)[0, 1]
    pred = int(prob >= threshold)
    return pred, float(prob)

def predict_svm(scaler, rbf, clf, X):
    Xf = X.reshape(1, -1)
    Xs = scaler.transform(Xf)
    Xr = rbf.transform(Xs)
    prob = clf.predict_proba(Xr)[0, 1]
    pred = int(prob >= 0.5)
    return pred, float(prob)

pos_ratio_threshold = 0.01

for i, path in enumerate(mini_test_paths):
    data = np.load(path)

    x_after = data["x_after"].astype(np.float32)
    y_mask  = data["y"]

    pos_ratio = (y_mask > 0).mean()
    y = 1 if pos_ratio >= pos_ratio_threshold else 0

    if x_after.ndim == 3 and x_after.shape[0] == 6:
        x_hw6 = np.moveaxis(x_after, 0, -1)
    elif x_after.ndim == 3:
        x_hw6 = x_after
    elif x_after.ndim == 4:
        x_tmp = x_after[0]
        if x_tmp.shape[0] == 6:
            x_hw6 = np.moveaxis(x_tmp, 0, -1)
        else:
            x_hw6 = x_tmp
    else:
        raise ValueError(f"شكل غير متوقع لـ x_after في {path}: {x_after.shape}")


    mu = x_hw6.mean(axis=(0, 1), keepdims=True)
    sd = x_hw6.std(axis=(0, 1), keepdims=True)
    x_std = (x_hw6 - mu) / (sd + 1e-8)

    X_cnn = x_std.reshape(1, 256, 256, 6)

    cnn_prob = float(cnn_model.predict(X_cnn, verbose=0)[0, 0])
    cnn_pred = int(cnn_prob >= 0.5)


    rf_pred,  rf_prob  = predict_rf(rf_model, x_hw6)
    svm_pred, svm_prob = predict_svm(svm_scaler, svm_rbf, svm_clf, x_hw6)

    print(f"Example {i+1}: {os.path.basename(path)}")
    print(f" - Truth label : {y}  (pos_ratio = {pos_ratio:.3f})")
    print(f" - CNN         : pred={cnn_pred}  prob={cnn_prob:.4f}")
    print(f" - RandomForest: pred={rf_pred}   prob={rf_prob:.4f}")
    print(f" - SVM         : pred={svm_pred}  prob={svm_prob:.4f}")
    print("-" * 60)

Found files:
 - /content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_0020_2019_01_16_0128_0256_A.npz OK
 - /content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_46_2019_02_23_0256_0128_A.npz OK
 - /content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_54_2019_03_14_0128_0000_A.npz OK
 - /content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_58_2019_04_14_0000_0256_A.npz OK
 - /content/drive/MyDrive/mini_test_5/sen12floods_s1_labels_0140_2019_03_13_0000_0000_A.npz OK
Example 1: sen12floods_s1_labels_0020_2019_01_16_0128_0256_A.npz
 - Truth label : 0  (pos_ratio = 0.000)
 - CNN         : pred=0  prob=0.0034
 - RandomForest: pred=0   prob=0.3848
 - SVM         : pred=1  prob=0.5149
------------------------------------------------------------
Example 2: sen12floods_s1_labels_46_2019_02_23_0256_0128_A.npz
 - Truth label : 1  (pos_ratio = 1.000)
 - CNN         : pred=1  prob=0.9379
 - RandomForest: pred=1   prob=0.6600
 - SVM         : pred=0  prob=0.4698
-------------------------